In [ ]:
import numpy as np
import pandas as pd
import ast


In [ ]:
movies=pd.read_csv(r"C:\Users\sagar\Downloads\tmdb_5000_movies.csv\tmdb_5000_movies.csv")
movies.shape

In [ ]:
credits=pd.read_csv(r"C:\Users\sagar\Downloads\tmdb_5000_credits.csv\tmdb_5000_credits.csv")
credits.shape

In [ ]:
movies.columns

In [ ]:
credits.columns

In [ ]:
movies=movies.merge(credits,on="title")
movies.columns

In [ ]:
movies=movies.drop(labels=["budget","homepage","original_language","original_title","popularity","production_companies","production_countries","release_date","revenue","runtime","spoken_languages","status","tagline","vote_average","vote_count","movie_id"],axis=1)

In [ ]:
movies.columns

In [ ]:
movies.head(1)

In [ ]:
movies.isnull().sum()

In [ ]:
movies=movies.dropna()

In [ ]:
movies.isnull().sum()

In [ ]:
movies.duplicated().sum()

In [ ]:
movies.iloc[0].genres

In [ ]:
movies.head(1)

In [ ]:
def convert(obj):
    l=[]
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

        


In [ ]:
movies['genres']=movies['genres'].apply(convert)


In [ ]:
movies['keywords']=movies['keywords'].apply(convert)


In [ ]:
movies.head(1)

In [ ]:
def convert_cast(obj):
    l=[]
    counter=0
    for i in ast.literal_eval(obj):
        if counter!=3:
          l.append(i['name'])
          counter=counter+1
    return l

In [ ]:
movies['cast']=movies['cast'].apply(convert_cast)

In [ ]:
movies.head(1)

In [ ]:
movies['crew'][0]

In [ ]:
def find_director(obj):
    l=[]
    
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            l.append(i['name'])
          
    return l

In [ ]:
movies['crew']=movies['crew'].apply(find_director)

In [ ]:
movies.head(1)

In [ ]:
movies['overview'][0]

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())


In [ ]:
movies=movies[["id","title","genres","keywords","overview","cast","crew"]]

In [ ]:
movies.head()

In [ ]:
movies["genres"]=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies["keywords"]=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies["cast"]=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])


In [ ]:
movies["crew"]=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags']=movies['genres']+movies['keywords']+movies['overview']+movies['cast']+movies['crew']

In [ ]:
movies.head()

In [ ]:
new_df=movies[["id","title","tags"]]
new_df.head()


In [ ]:
new_df["tags"]=new_df["tags"].apply(lambda x:" ".join(x))

In [ ]:
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags']=new_df['tags'].apply(lambda x:x.lower())

In [ ]:
new_df.head()

# VECTORIZATION

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

In [ ]:
cv=CountVectorizer(max_features=5000,stop_words="english")

In [ ]:
vectors=cv.fit_transform(new_df['tags']).toarray()
vectors


In [ ]:
vectors[0]

In [ ]:
cv.get_feature_names_out()

In [ ]:
!pip install nltk

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
def stem(text):
    l=[]
    for i in text.split():
        l.append(ps.stem(i))
    return " ".join(l)

In [ ]:
new_df['tags']=new_df['tags'].apply(stem)
new_df['tags']

In [ ]:
cv.get_feature_names_out()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)
similarity

In [ ]:
new_df[new_df['title']=='Avatar'].index[0]

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:11]

In [ ]:
def recommend(movie):
    movie_index=new_df[new_df['title']==movie].index[0]
    distances=similarity[movie_index]
    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:11]
    
    for i in movies_list:
        print(new_df.iloc[i[0]].title)
    

In [ ]:
recommend('Avatar')

In [ ]:
recommend('Batman Begins')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df,open('movies.pkl','wb'))

In [ ]:
new_df.to_dict() 

In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))